## Описание

Статистика фигурного катания

## Импорты

In [2]:
pip install beautifulsoup4 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install lxml


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

## Получение данных

В качестве источника взят сайт с официальными результатами соревнований по фигруному катанию в новой системе оценивания

Общий вид страницы выглядит следующим образом:

https://skatingscores.com/2425/wc/sr/women/i/short/usa/alysa_liu/

https://skatingscores.com/сезон/чемпионат/возрастная категория/дисциплина/i/программа/страна/участник/

Соберем информацию для чемпионатов мира за все года с новой системой оценивания в взрослом женском одиночном катании. Для этого нужно получить список всех имен участниц



In [5]:
# Общая страница с результатами коротной программы сезона 24-25
url = 'https://skatingscores.com/2425/wc/sr/women/i/short/'
# Начало ссылки для участницы
base_link = 'https://skatingscores.com/2425/wc/sr/women/i/short/'
# Проверяем доступ 
page = requests.get(url)
print(page.status_code)
# Загружаем данные
soup = BeautifulSoup(page.text, "html.parser")

200


In [6]:
# Получаем все ссылки
for link in soup.find_all('a'):
    link_ = link.get('href')
    # Фильтруем ссылки участниц от ссылок на судей и прочих возможных по шаблону
    if '/re/2425/wc/sr/women/i/short' in link_:
        country_girl = link_.split('/')[-3]
        girl = link_.split('/')[-2]
        # Собираем ссылку
        girl_link = base_link+'/'+country_girl+'/'+girl+'/'
        print(girl_link)
        page_girl = requests.get(girl_link)
        print(page_girl.status_code)    
        soup_girl = BeautifulSoup(page_girl.text, "html.parser")
        time.sleep(5)
        break


https://skatingscores.com/2425/wc/sr/women/i/short//usa/alysa_liu/
200


In [ ]:
# Получаем все таблицы со страницы
tables = soup_girl.find_all('table')

In [ ]:
## Все таблицы надо привести к формату для анализа, оставив только нужные показатели

# В итоге должно получится две таблицы - в разрезе общих результатов и подробно по элементам

# В первой таблице стартовый номер, дедакшин и тотал
df_1 = pd.read_html(str(tables[0]))[0].tail(1).rename(columns={1: 'Спортсменка', 2: 'Страна', 3: 'Стартовый номер', 4: 'Дедакшн', 5: 'Итоговый результат'})
df_1


C:\Users\пользователь\AppData\Local\Temp\ipykernel_19968\1545850799.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_1 = pd.read_html(str(tables[0]))[0].tail(1).rename(columns={1: 'Спортсменка', 2: 'Страна', 3: 'Стартовый номер', 4: 'Дедакшн', 5: 'Итоговый результат'})


,0,Спортсменка,Страна,Стартовый номер,Дедакшн,Итоговый результат
1,1,Alysa LIU21,USA,18,0.00,PB 74.58


In [ ]:
# Во второй TSS судей
df_2 = pd.read_html(str(tables[1]))[0]    
df_2 = df_2.drop(columns=[i for i  in range(6)]+[15]).rename(columns=df_2.iloc[0]).tail(1)
# Убираем лишние цифры
df_2  = df_2 .replace(to_replace ='\d{1}\s', value = '', regex = True)
df_2                     


C:\Users\пользователь\AppData\Local\Temp\ipykernel_19968\1241738416.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_2 = pd.read_html(str(tables[1]))[0]


,ISR 🇮🇱,SVK 🇸🇰,CZE 🇨🇿,FRA 🇫🇷,NOR 🇳🇴,SUI 🇨🇭,JPN 🇯🇵,ITA 🇮🇹,ROU 🇷🇴
1,77.23,75.71,73.06,72.85,74.68,73.65,73.92,75.45,74.65


In [46]:
pd.read_html(str(tables[10]))[0]

C:\Users\пользователь\AppData\Local\Temp\ipykernel_19968\717955009.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(tables[10]))[0]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),PB 4 33.78


In [58]:
df_3 = pd.read_html(str(tables[3]))[0][[0,1]].T
df_3 = df_3.rename(columns=df_3.iloc[0]).tail(1)
df_3

C:\Users\пользователь\AppData\Local\Temp\ipykernel_19968\8891660.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_3 = pd.read_html(str(tables[3]))[0][[0,1]].T


,BVSB 2 32.59,BV,GOE,Jumps,Spins,TES
1,GOESB 1 8.21,SB 2 32.59,SB 1 8.21,1 22.76,1 12.75,SB 1 40.80


In [128]:
pd.read_html(str(tables[4]))[0]

C:\Users\пользователь\AppData\Local\Temp\ipykernel_10668\3962321705.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(tables[4]))[0]


,0,1
0,BV,SB 2 32.59


In [129]:
pd.read_html(str(tables[5]))[0]

C:\Users\пользователь\AppData\Local\Temp\ipykernel_10668\3683640326.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(tables[5]))[0]


,0,1
0,GOE,SB 1 8.21


In [130]:
pd.read_html(str(tables[6]))[0]

C:\Users\пользователь\AppData\Local\Temp\ipykernel_10668\248377429.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(tables[6]))[0]


,0,1
0,Jumps,1 22.76


In [131]:
pd.read_html(str(tables[7]))[0]

C:\Users\пользователь\AppData\Local\Temp\ipykernel_10668\4217466188.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(tables[7]))[0]


,0,1
0,Spins,1 12.75


In [132]:
pd.read_html(str(tables[8]))[0]

C:\Users\пользователь\AppData\Local\Temp\ipykernel_10668\2548042947.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(tables[8]))[0]


,0,1
0,TES,SB 1 40.80


In [ ]:
# Элементы подробно

# ## Третья - оценки за элементы по технике
# df_3 = pd.read_html(str(tables[2]))[0]

# pd.read_html(str(tables[9]))[0]

C:\Users\пользователь\AppData\Local\Temp\ipykernel_10668\717955009.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(tables[10]))[0]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),Program Component Score (factored),PB 4 33.78
